## Merge toute les données intérim (variables des 4 piliers)

In [51]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.3f}'.format

external_data = Path('../data/external/')
processed_data = Path('../data/processed/')
raw_data = Path('../data/raw/')
interim_data = Path('../data/interim/')

In [27]:
!ls -lh ../data/interim/

total 9,0M
-rw-rw-r-- 1 outini outini 822K nov.  28 00:23 acces_point_phys_service_publique.csv
-rw-rw-r-- 1 outini outini  23K nov.  27 01:00 data_merge_V2.ipynb
-rw-rw-r-- 1 outini outini 4,0M nov.  28 00:23 data_scol_menage_demandeurEmplois.csv
-rw-rw-r-- 1 outini outini  79K nov.  27 01:00 IFN_var_scol_menage_demandeEmplois.ipynb
-rw-rw-r-- 1 outini outini 837K nov.  28 00:23 taux_couverture_mobile.csv
-rw-rw-r-- 1 outini outini 825K nov.  28 00:23 taux_couverture_THD.csv
-rw-rw-r-- 1 outini outini 706K nov.  28 00:23 taux_demandeurs_emploi.csv
-rw-rw-r-- 1 outini outini 781K nov.  28 00:23 taux_pauvrete.csv
-rw-rw-r-- 1 outini outini 999K nov.  28 00:23 variable_minimas_V2.csv


In [32]:
df1 = pd.read_csv(interim_data/'acces_point_phys_service_publique.csv')
df2 = pd.read_csv(interim_data/'data_scol_menage_demandeurEmplois.csv')
df3 = pd.read_csv(interim_data/'taux_couverture_mobile.csv')
df4 = pd.read_csv(interim_data/'taux_couverture_THD.csv')
df5 = pd.read_csv(interim_data/'taux_demandeurs_emploi.csv')    # get TX_DEMANDEUR_EMPLOIS from this one
df6 = pd.read_csv(interim_data/'taux_pauvrete.csv')
df7 = pd.read_csv(interim_data/'variable_minimas_V2.csv')

communes = pd.read_csv(raw_data/'table_insee_libcom_dep.csv').drop(columns=['Unnamed: 0'])

### Check pour avoir meme base de 3 index : (CODE_INSEE, LIBCOM, DEP) et meme taille de dataset

In [33]:
for data in [df1, df2, df3, df4, df5, df6, df7]:
    print(data.columns)
    print(data.shape)
    print()

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'ACCES_SERVICE_PUBLIC'], dtype='object')
(35013, 4)

Index(['Unnamed: 0', 'CODE_INSEE', 'LIBCOM', 'DEP', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS', 'TX_DEMANDEURS_EMPLOI'], dtype='object')
(36101, 10)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'COUVERTURE_MOBILE'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'DEP', 'LIBCOM', 'TAUX_COUVERTURE_THD'], dtype='object')
(35359, 4)

Index(['CODE_INSEE', 'LIBCOM', 'TX_DEMANDEUR_EMPLOIS'], dtype='object')
(35010, 3)

Index(['CODE_INSEE', 'LIBCOM', 'TX_POVERTY'], dtype='object')
(35010, 3)

Index(['Unnamed: 0', 'CODE_INSEE', 'LIBCOM', 'DEP', 'TX_RSA'], dtype='object')
(35010, 5)



### Check nouvelle taille de chaque dataset

In [34]:
indexes = ['CODE_INSEE', 'LIBCOM', 'DEP']
how = 'left'

df1 = communes.merge(df1, how=how, on=indexes).drop_duplicates(subset=indexes)
df2.drop(columns=['Unnamed: 0', 'TX_DEMANDEURS_EMPLOI'], inplace=True)
df2 = communes.merge(df2, how=how, on=indexes).drop_duplicates(subset=indexes)
df3 = communes.merge(df3, how=how, on=indexes).drop_duplicates(subset=indexes)
df4 = communes.merge(df4, how=how, on=indexes).drop_duplicates(subset=indexes)
df5 = communes.merge(df5, how=how, on=['CODE_INSEE', 'LIBCOM']).drop_duplicates(subset=indexes)
df6 = communes.merge(df6, how=how, on=['CODE_INSEE', 'LIBCOM']).drop_duplicates(subset=indexes)
df7.drop(columns=['Unnamed: 0'], inplace=True)

for data in [df1, df2, df3, df4, df5, df6, df7]:
    print(data.columns)
    print(data.shape)
    print()

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'ACCES_SERVICE_PUBLIC'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS'], dtype='object')
(35010, 8)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'COUVERTURE_MOBILE'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TAUX_COUVERTURE_THD'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_DEMANDEUR_EMPLOIS'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_POVERTY'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_RSA'], dtype='object')
(35010, 4)



### MERGE

In [133]:
df = df1.merge(df2, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df3, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df4, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df5, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df6, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df7, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])

print(df.shape)

(35010, 14)


In [134]:
df

,CODE_INSEE,LIBCOM,DEP,ACCES_SERVICE_PUBLIC,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,COUVERTURE_MOBILE,TAUX_COUVERTURE_THD,TX_DEMANDEUR_EMPLOIS,TX_POVERTY,TX_RSA
0,01001,L'Abergement-Clémenciat,01,0,52.154,22.824,4.832,23.779,11.309,nan,1.000,nan,nan,nan
1,01002,L'Abergement-de-Varey,01,0,40.426,28.571,4.762,23.404,7.979,1.000,1.000,nan,nan,nan
2,01004,Ambérieu-en-Bugey,01,0,49.475,41.515,10.216,20.734,16.620,1.000,1.000,21.755,26.171,2.594
3,01005,Ambérieux-en-Dombes,01,1,53.575,21.076,6.602,19.488,13.854,1.000,1.000,nan,nan,nan
4,01006,Ambléon,01,0,53.061,36.364,9.091,26.531,10.204,0.650,1.000,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35005,97613,M'Tsangamouji,976,0,nan,nan,nan,nan,nan,nan,1.000,nan,nan,nan
35006,97614,Ouangani,976,0,nan,nan,nan,nan,nan,nan,0.000,nan,nan,nan
35007,97615,Pamandzi,976,1,nan,nan,nan,nan,nan,nan,1.000,nan,51.412,nan
35008,97616,Sada,976,1,nan,nan,nan,nan,nan,nan,1.000,nan,49.199,nan


### Cela résulte en 11 colonnes pour calculer le score à savoir: 

In [122]:
list(df.drop(columns=indexes).columns)

['ACCES_SERVICE_PUBLIC',
 'TX_NSCOL15P',
 'TX_MENSEUL',
 'TX_FAMMONO',
 'TX_65ETPLUS',
 'TX_25ETMOINS',
 'COUVERTURE_MOBILE',
 'TAUX_COUVERTURE_THD',
 'TX_DEMANDEUR_EMPLOIS',
 'TX_POVERTY',
 'TX_RSA']

### + 3 colonnes d'index :

In [123]:
list(indexes)

['CODE_INSEE', 'LIBCOM', 'DEP']

In [124]:
df.to_csv(processed_data/'MERGE_data.csv', index=False)

### Check missing values (NaN --> median par département)

In [135]:
# Pourcentage des valeurs manquantes par colonne
df_na = df.isna().sum().sort_values(ascending=False)
df_na = df_na / len(df) * 100
df_na

TX_RSA                 97.181
TX_DEMANDEUR_EMPLOIS   96.638
TX_POVERTY             83.973
COUVERTURE_MOBILE      52.856
TAUX_COUVERTURE_THD     0.131
TX_FAMMONO              0.089
TX_MENSEUL              0.089
TX_25ETMOINS            0.069
TX_65ETPLUS             0.069
TX_NSCOL15P             0.069
ACCES_SERVICE_PUBLIC    0.000
DEP                     0.000
LIBCOM                  0.000
CODE_INSEE              0.000
dtype: float64

### Remplir les NaN par médiane par dép.

In [136]:
for col in df.drop(columns=indexes).columns:
    df[col] = df[col].fillna(df.groupby('DEP')[col].transform('median'))

#df['TX_POVERTY'] = df['TX_POVERTY'].fillna(df.groupby('DEP')['TX_POVERTY'].agg('median'))

In [137]:
# Pourcentage des valeurs manquantes par colonne
df_na = df.isna().sum().sort_values(ascending=False)
df_na = df_na / len(df) * 100
df_na

COUVERTURE_MOBILE      0.368
TX_RSA                 0.300
TX_DEMANDEUR_EMPLOIS   0.300
TAUX_COUVERTURE_THD    0.057
TX_25ETMOINS           0.049
TX_65ETPLUS            0.049
TX_FAMMONO             0.049
TX_MENSEUL             0.049
TX_NSCOL15P            0.049
TX_POVERTY             0.000
ACCES_SERVICE_PUBLIC   0.000
DEP                    0.000
LIBCOM                 0.000
CODE_INSEE             0.000
dtype: float64

#### Il reste quelques données manquantes au niveau de DOMTOM sur la couverture mobile par ex. (en cours ...)

In [138]:
df.to_csv(processed_data/'MERGE_data_clean.csv', index=False)